In [1]:
import gzip

In [8]:
# Get all mutations
class Mutaton:
    def __init__(self, acc, pos, wtAA, mutAA, mut_type):
        self.acc = acc
        self.pos = pos
        self.wtAA = wtAA
        self.mutAA = mutAA
        self.mut_type = mut_type
        self.prediction = None
        self.prob = None
        self.predictions = {}
    
    def show(self):
        print (self.acc, self.pos, self.wtAA, self.mutAA, self.mut_type, self.prediction, self.prob)


In [9]:
dic_mutations = {}
for line in open('../ML/final_ADRN.txt'):
    acc = line.split()[0].split('/')[0]
    mutation = line.split()[0].split('/')[1]
    name = line.split()[0]
    pos = mutation[1:-1]
    wtAA = mutation[0]
    mutAA = mutation[-1]
    mut_type = line.split()[1].strip()
    dic_mutations[name] = Mutaton(acc, pos, wtAA, mutAA, mut_type)


In [11]:
# get all the predictions from the PolyPhen2 output file
predictor = 'polyphen2'
with gzip.open('polyphen_output_new_test_set.tsv.gz', 'rt') as f:
    for line in f:
        if line.startswith('#'):
            continue
        else:
            line = line.strip().split('\t')
            acc = line[0].strip()
            pos = line[1].strip()
            wtAA = line[2].strip()
            mutAA = line[3].strip()
            name = acc + '/' + wtAA+ pos + mutAA
            prediction = line[9].strip()
            prob = line[10].strip()
            # print (name, acc, pos, wtAA, mutAA, prediction)
            if name in dic_mutations:
                if predictor not in dic_mutations[name].predictions:
                    dic_mutations[name].predictions[predictor] = {}
                dic_mutations[name].predictions[predictor]['prediction'] = prediction
                dic_mutations[name].predictions[predictor]['prob'] = prob
            else:
                print (f'{name} not found in the DB')

print (dic_mutations['O60674/H608Y'].predictions)

{'polyphen2': {'prediction': 'probably damaging', 'prob': '1'}}


In [ ]:
predictor = 'pmut'
with gzip.open('pmut_output.tsv.gz', 'rt') as f:
    for line in f:
        acc = line.split()[0]
        mutation = line.split()[1]
        pos = mutation[1:-1]
        wtAA = mutation[0]
        mutAA = mutation[-1]
        name = acc + '/' + wtAA+ pos + mutAA
        prediction = 'True' if  'True' in line.split()[2] else 'False'
        prob = line.split()[2].split('(')[1].split(')')[0]
        # print (name, acc, pos, wtAA, mutAA, prediction)
        if name in dic_mutations:
            if predictor not in dic_mutations[name].predictions:
                dic_mutations[name].predictions[predictor] = {}
            dic_mutations[name].predictions[predictor]['prediction'] = prediction
            dic_mutations[name].predictions[predictor]['prob'] = prob
        else:
            print (f'{name} not found in the DB')

print (dic_mutations['O60674/H608Y'].predictions)

In [ ]:

MUT_TYPES = {
            'A': ['activating', 'increase'],
            'D': ['decrease', 'loss'],
            'R': ['resistance'],
            'T': ['activatingresistance'],
            'N': ['neutral']
            }
text = ''
for mut_type in MUT_TYPES:
    names = []
    y_pred = []; y_true = []; y_prob=[]; y_act_deact_or_neutral = []
    for mutation in dic_mutations:
        if dic_mutations[mutation].mut_type in ['activatingresistance', 'increaseresistance']:
            print (mutation, dic_mutations[mutation].mut_type, dic_mutations[mutation].prob)
        if dic_mutations[mutation].prediction is None: continue
        kinase_mut_type = dic_mutations[mutation].mut_type
        if kinase_mut_type not in MUT_TYPES[mut_type]: continue
        # print (mutation, dic_mutations[mutation].mut_type, dic_mutations[mutation].prediction)
        if 'damaging' in dic_mutations[mutation].prediction:
            y_pred.append(1)
        else:
            y_pred.append(0)
        y_prob.append(float(dic_mutations[mutation].prob))
        if kinase_mut_type in MUT_TYPES['N']:
            y_act_deact_or_neutral.append(0)
        else:
            y_act_deact_or_neutral.append(1)
        if kinase_mut_type in MUT_TYPES[mut_type]:
            y_true.append(1)
        else:
            y_true.append(0)
        names.append(mutation)
    # print (f'MCC: {matthews_corrcoef(y_true, y_pred)}')
    print (mut_type, len(y_pred), f'REC: {recall_score(y_true, y_pred)}')
    # print (f'AUC: {roc_auc_score(y_true, y_pred)}')
    if mut_type == 'T':
        print (y_true)
        print (y_pred)

    for name, y_p, y_t in zip(names, y_prob, y_act_deact_or_neutral):
        # print (name, y_p, y_t)
        if 'V600E' in name: dic_mutations[name].show()
        text += str(name) + '\t' + str(dic_mutations[name].mut_type) + '\t' + str(y_p) + '\t' + str(y_t) + '\n'
